In [1]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy.linalg import fractional_matrix_power
from IAGCWD import image_agcwd, process_bright, process_dimmed


cap = cv2.VideoCapture('videos/Laguna Beach.webm')
fps = cap.get(cv2.CAP_PROP_FPS)
vid_width = 720

start_sec = 120
end_sec = 140
start_frame = fps * start_sec
end_frame = fps * end_sec
frame_count = {'all':0, 'total':0, 'bright':0, 'dim':0, 'neutral':0}

threshold = 0.3
T_t = 112 # Global average brightness 

cv2.namedWindow('Video')


while cap.isOpened():
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret or frame_count['all'] > end_frame:
        print("Can't receive frame. Exiting ...")
        break
    frame_count['all'] += 1
    if frame_count['all'] < start_frame:
        continue
    frame_count['total'] += 1
    
    # resize
    dim = (vid_width, int(vid_width*frame.shape[0]/frame.shape[1]))
    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    
    # agc algorithm
    YCrCb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    Y = YCrCb[:,:,0]
    M,N = frame.shape[:2]
    m_I = np.sum(Y/(M*N)) 
    t = (m_I - T_t)/ T_t

    img_output = frame
    if t < -threshold:
        frame_count['dim'] += 1
        result = process_dimmed(Y)
        YCrCb[:,:,0] = result
        img_output = cv2.cvtColor(YCrCb,cv2.COLOR_YCrCb2BGR)
    elif t > threshold:
        frame_count['bright'] += 1
        result = process_bright(Y)
        YCrCb[:,:,0] = result
        img_output = cv2.cvtColor(YCrCb,cv2.COLOR_YCrCb2BGR)
    else:
        frame_count['neutral'] += 1
    
    
    img = cv2.hconcat([frame, img_output])
    cv2.imshow('Video', img)
    
    if cv2.waitKey(1)  == ord('q'):
        break

for key in frame_count:
    if key == 'all':
        continue
    print(f'{key} frames: {frame_count[key]}')

cap.release()
cv2.destroyAllWindows()

total frames: 418
bright frames: 27
dim frames: 0
neutral frames: 391
